<a href="https://colab.research.google.com/github/GuillermoCarbajal/J-MOPD/blob/main/J-MOPD_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Joint Motion Offset Prediction and Deblurring (J-MOPD) demo

## Clone the repo

In [5]:
!git clone https://github.com/GuillermoCarbajal/J-MOPD.git
%cd J-MOPD

Cloning into 'J-MOPD'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 42 (delta 8), reused 30 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 1.09 MiB | 21.46 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/J-MOPD/J-MOPD


## Setup the environment

In [6]:
!pip install -r 'requirements.txt'

## Download pretrained models

In [7]:
import gdown
gdown.download('http://iie.fing.edu.uy/~carbajal/J-MOPD/crop320/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27.pkl', "./pretrained_models/", quiet=False)
gdown.download('http://iie.fing.edu.uy/~carbajal/J-MOPD/crop320/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27_G.pkl', "./pretrained_models/", quiet=False)

Downloading...
From: http://iie.fing.edu.uy/~carbajal/J-MOPD/crop320/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27.pkl
To: /content/J-MOPD/J-MOPD/pretrained_models/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27.pkl
100%|██████████| 4.69M/4.69M [00:01<00:00, 2.47MB/s]
Downloading...
From: http://iie.fing.edu.uy/~carbajal/J-MOPD/crop320/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27_G.pkl
To: /content/J-MOPD/J-MOPD/pretrained_models/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27_G.pkl
100%|██████████| 68.1M/68.1M [00:07<00:00, 8.81MB/s]


'./pretrained_models/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27_G.pkl'

In [9]:
import torch
from skimage.io import imread
from skimage.transform import rescale
from utils.homographies import compute_intrinsics, generate_video, get_offsets_from_positions, \
                               save_kernels_from_offsets, show_kernels_from_offsets_on_blurry_image, save_motion_flow, reblur_offsets
import numpy as np
#from models.network_nimbusr_pmbm import NIMBUSR_PMBM as net
from models.network_nimbusr_offsets import NIMBUSR_Offsets as net_nimbusr_offsets

from utils.visualization import save_image, tensor2im, save_video, sort_positions, show_positions_found
import os
import argparse
from models.CameraShakeModel_OffsetNet import OffsetNet_quad as OffsetsNet
import json
from PIL import Image
import cv2

## Offsets Computation

In [17]:
#input_image = 'testing_imgs/0316.png'
input_image ='testing_imgs/manmade_01_gyro_01.png'
camera_model_file = 'pretrained_models/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27.pkl'

n_positions= 25 # number of positions estimated by the network
if torch.cuda.is_available():
  offsets_net = OffsetsNet(n_offset=n_positions,offset_mode='raw').cuda()
  state_dict = torch.load(camera_model_file )
else:
  offsets_net = OffsetsNet(n_offset=n_positions,offset_mode='raw').to('cpu')
  state_dict = torch.load(camera_model_file, map_location=torch.device('cpu') )
  print('cuda is not available')


offsets_net.load_state_dict(state_dict)
offsets_net.eval()

cuda is not available


OffsetNet_quad(
  (pad_1): ReflectionPad2d((1, 2, 1, 2))
  (todepth): SpaceToDepth()
  (conv_1): Sequential(
    (0): ReflectionPad2d((2, 2, 2, 2))
    (1): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
    (2): ReLU(inplace=True)
    (3): Sequential(
      (0): ResnetBlock_woNorm(
        (conv_block): Sequential(
          (0): ReplicationPad2d((2, 2, 2, 2))
          (1): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
          (2): ReLU(inplace=True)
          (3): ReplicationPad2d((2, 2, 2, 2))
          (4): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
        )
      )
      (1): ResnetBlock_woNorm(
        (conv_block): Sequential(
          (0): ReplicationPad2d((2, 2, 2, 2))
          (1): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
          (2): ReLU(inplace=True)
          (3): ReplicationPad2d((2, 2, 2, 2))
          (4): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
        )
      )
      (2): ResnetBlock_woNorm(
        (conv_block): Sequential(
 

In [18]:
def compute_offsets(input_file, offsets_net, output_dir='testing_results',
                       gamma_factor=1.0, save_video=False):

  n_positions= 25 # number of positions estimated by the network
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  img_name = input_file.split('/')[-1]
  img_name, ext = img_name.split('.')

  print('loading image ',input_file)
  blurry_image = imread(input_file)/255.0

  # Blurry image is transformed to pytorch format
  if torch.cuda.is_available():
    blurry_tensor = torch.from_numpy(blurry_image).permute(2,0,1)[None].cuda().float()
  else:
    blurry_tensor = torch.from_numpy(blurry_image).permute(2,0,1)[None].float()

  _, C,H,W = blurry_tensor.shape

  # Kernels and masks are estimated
  blurry_tensor_to_compute_kernels = blurry_tensor**gamma_factor - 0.5


  with torch.no_grad():
      blurry_tensor_ph = blurry_tensor**gamma_factor
      offsets = offsets_net(blurry_tensor_ph - 0.5)
      offsets = offsets[:,:,:H,:W]
      blurry_tensor = blurry_tensor[:,:,:H,:W]


  offsets = offsets.reshape(1,2*n_positions, H,W)
  offsets_BT=None


  kernels_file = os.path.join(output_dir, img_name + '_kernels_found.png' )


  show_kernels_from_offsets_on_blurry_image(blurry_tensor[0],offsets[0].reshape(n_positions,2,H,W), kernels_file)
  print('Kernels saved in ',os.path.join(output_dir, img_name + '_kernels.png') )


In [19]:
compute_offsets(input_image, offsets_net)

loading image  testing_imgs/manmade_01_gyro_01.png


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

## Image Deblurring

In [ ]:
#from models.network_nimbusr_pmbm import NIMBUSR_PMBM as net
from models.network_nimbusr_offsets import NIMBUSR_Offsets as net_nimbusr_offsets

In [ ]:
def load_nimbusr_net(type='offsets'):
    opt_net = { "n_iter": 8
        , "h_nc": 64
        , "in_nc": 4 #2 if args.gray else 4 #4
        , "out_nc":3 #1 if args.gray else 3 #3
        #, "ksize": 25
        , "nc": [64, 128, 256, 512]
        , "nb": 2
        , "gc": 32
        , "ng": 2
        , "reduction" : 16
        , "act_mode": "R"
        , "upsample_mode": "convtranspose"
        , "downsample_mode": "strideconv"}

    path_pretrained = args.restoration_network #r'../model_zoo/NIMBUSR.pth'

    if type=='pmbm':
        netG = net(n_iter=opt_net['n_iter'],
                    h_nc=opt_net['h_nc'],
                    in_nc=opt_net['in_nc'],
                    out_nc=opt_net['out_nc'],
                    nc=opt_net['nc'],
                    nb=opt_net['nb'],
                    act_mode=opt_net['act_mode'],
                    downsample_mode=opt_net['downsample_mode'],
                    upsample_mode=opt_net['upsample_mode']
                    )
    elif type=='offsets':
        netG = net_nimbusr_offsets(n_iter=opt_net['n_iter'],
            h_nc=opt_net['h_nc'],
            in_nc=opt_net['in_nc'],
            out_nc=opt_net['out_nc'],
            nc=opt_net['nc'],
            nb=opt_net['nb'],
            act_mode=opt_net['act_mode'],
            downsample_mode=opt_net['downsample_mode'],
            upsample_mode=opt_net['upsample_mode']
            )

    if os.path.exists(path_pretrained):
        print('Loading model for G [{:s}] ...'.format(path_pretrained))
        netG.load_state_dict(torch.load(path_pretrained))
    else:
        print('Model does not exists')

    netG = netG.to('cuda')

    return netG


In [ ]:
def restore_images(input_file, reblur_model, output_folder='testing_results', gamma_factor=1.0,
                   resize_factor=1, network_type='nimbusr_sat', focal_length=-1, save_video=False):

  n_positions= 25 # number of positions estimated by the network
  if not os.path.exists(output_folder):
      os.makedirs(output_folder)

  img_name = input_file.split('/')[-1]
  img_name, ext = img_name.split('.')

  print('loading image ',input_file)
  blurry_image = imread(input_file)

  # Blurry image is transformed to pytorch format
  transform = transforms.Compose([
      transforms.ToTensor()
  ])
  blurry_tensor = transform(blurry_image).cuda()

  _, C,H,W = blurry_tensor.shape

  # Kernels and masks are estimated
  blurry_tensor_to_compute_kernels = blurry_tensor**gamma_factor - 0.5


  if focal_length > 0:
      f = torch.Tensor([focal_length]).to(blurry_tensor.device)
      #f = torch.Tensor([float(max(H,W))]).to(tensor_img.device)
      intrinsics = torch.Tensor([[f, 0, W/2],[0, f, H/2], [0, 0, 1] ]).cuda(blurry_tensor.device)
      intrinsics = intrinsics[None,:,:]
  else:
      intrinsics = compute_intrinsics(W,H).cuda(blurry_tensor.device)[None]
      f =  torch.Tensor([max(H,W)]).to(blurry_tensor.device)

      #focal_channel = f/maximo * torch.ones(N,1,H,W).to(tensor_img.device)
      #cam_input = torch.concat((focal_channel, tensor_img), dim=1)

  # get positions
  camera_positions = camera_model(blurry_tensor - 0.5,f)

  order = sort_positions(camera_positions[0])
  camera_positions[0] = camera_positions[0,order,:]

  offsets = get_offsets_from_positions(blurry_tensor.shape, camera_positions, intrinsics)
  offsets = offsets.reshape(1,2*n_positions, H,W)
  offsets_BT=None


  netG = load_nimbusr_net('offsets')
  netG.eval()
  noise_level = 0.01
  noise_level = torch.FloatTensor([noise_level]).view(1,1,1).cuda(blurry_tensor.device)

  output = netG(blurry_tensor, offsets, 1, sigma=noise_level[None,:], offsets_BT=offsets_BT)


  output_img = tensor2im(torch.clamp(output[0].detach(),0,1) - 0.5)
  save_image(output_img, os.path.join(output_folder, img_name + '_PMBM.png' ))

  found_positions_np = camera_positions[0].detach().cpu().numpy()
  np.savetxt(os.path.join(output_folder,f'{img_name}_found_positions.txt'), found_positions_np)
  pose = np.zeros((found_positions_np.shape[0], 6))
  pose[:, 3:] = found_positions_np

  #K, _ = generarK((H,W,C), pose, A=intrinsics[0].detach().cpu().numpy())
  kernels_file = os.path.join(output_folder, img_name + '_kernels_found.png' )
  #kernels_estimated = mostrar_kernels(K, (H,W,C), output_name = kernels_file)


  show_kernels_from_offsets_on_blurry_image(blurry_tensor[0],offsets[0].reshape(n_positions,2,H,W), kernels_file)
  print('Kernels saved in ',os.path.join(output_folder, img_name + '_kernels.png') )

  save_image((255*blurry_image).astype(np.uint8), os.path.join(output_folder, img_name + '.png' ))
  print('Output saved in ', os.path.join(output_folder, img_name + '_J-MTPD.png' ))

  frames, reblurred = generate_video(output, camera_positions, intrinsics[0])
  print(reblurred.shape)
  reblurred = tensor2im(torch.clamp(reblurred[0].detach(),0,1) - 0.5)
  save_image(reblurred, os.path.join(output_folder, img_name + '_reblurred.png' ))

  show_positions_found(found_positions_np, intrinsics[0,0,0].detach().cpu().numpy(), os.path.join(output_folder, img_name + '_positions_found.png'))

  if save_video:
      #imgs=[];
      output_video = os.path.join(output_folder, img_name + '.avi')
      save_video(frames, output_video)
      print('Video saved in ', output_video)


In [ ]:
input_image = 'testing_imgs/0316.png'
input_image = 'testing_imgs/Blurry2_1.png'
restoration_model='pretrained_models/80000_G.pth'

In [ ]:
restore_images(input_image, camera_model)

## Try with your own images

In [ ]:
from google.colab import files
import shutil

upload_folder = 'upload/input'
result_folder = 'upload/output'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(upload_folder)
os.makedirs(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

In [ ]:
import glob
input_list = sorted(glob.glob(os.path.join(upload_folder, '*')))
for input_path in input_list:
  img_input = imread(input_path)
  restore_images(input_path, camera_model)